### 로지스틱 회귀

In [2]:
import pandas as pd
fish = pd.read_csv('data/Fish.csv')
fish.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


In [3]:
fish['Species'].unique()

array(['Bream', 'Roach', 'Whitefish', 'Parkki', 'Perch', 'Pike', 'Smelt'],
      dtype=object)

In [4]:
fish_input = fish[['Weight', 'Length2', 'Length3', 'Height', 'Width']]
fish_target = fish[['Species']]

In [5]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state=42)

In [6]:
# 데이터 표준화 해주는 클래스
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input) # ss.fit은 train_input 의 평균과 표준편차 구해줌 -> ss가 그 값을 들어있음
train_scaled = ss.transform(train_input) # ss.fit에서 구해준걸 train_input에 적용하여 표준화해줌
test_scaled = ss.transform(test_input)
# train_sclaed[:5]

In [110]:
# k최근접 알고리즘 활용해보기
# 비교분석
from sklearn.neighbors import KNeighborsClassifier

In [111]:
kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)
# kn.score(train_scaled, train_target)
kn.score(test_scaled, test_target)

C:\Users\2-22\Desktop\DMF\ML\venv\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.85

In [112]:
kn.classes_

array(['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish'],
      dtype=object)

In [113]:
test_sclead[:5]

array([[-0.88741352, -0.91804565, -1.03098914, -1.03098914, -0.90464451,
        -0.80762518],
       [-1.06924656, -1.50842035, -1.54345461, -1.54345461, -1.58849582,
        -1.93803151],
       [-0.54401367,  0.35641402,  0.30663259,  0.30663259, -0.8135697 ,
        -0.65388895],
       [-0.34698097, -0.23396068, -0.22320459, -0.22320459, -0.11905019,
        -0.12233464],
       [-0.68475132, -0.51509149, -0.58801052, -0.58801052, -0.8998784 ,
        -0.50124996]])

In [117]:
kn.predict(test_scaled[:5])

array(['Perch', 'Smelt', 'Pike', 'Perch', 'Perch'], dtype=object)

In [119]:
kn.predict_proba(test_scaled[:5])
# 위 결과를 유추 하기까지의 과정을 보여줌
# 0, 0, 1, 0, 0 -> pike 가 3마리 있다
# 0, 0, 0.66666667, 0, 0.333333 -> Pike가 2마리, Perch가 1마리

array([[0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.66666667, 0.        , 0.33333333,
        0.        , 0.        ],
       [0.        , 0.        , 0.66666667, 0.        , 0.33333333,
        0.        , 0.        ]])

In [120]:
# 이제 진짜 로지스틱 회귀 사용해보기
# 로지스틱 회귀 라고는 하지만 그냥 분류하는 법 이라고 생각하면 됨.
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
# 브림과 스멜트만 사용
train_bream_smelt = train_sclaed[bream_smelt_indexes['Species']]
target_brean_smelt = train_target[bream_smelt_indexes['Species']]

In [121]:
from sklearn.linear_model import LogisticRegression

In [122]:
lr = LogisticRegression()
lr.fit(train_bream_smelt, target_brean_smelt)
# numpy 랑 pandas 혼용 사용으러 워링 나옴

C:\Users\2-22\Desktop\DMF\ML\venv\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [123]:
lr.predict(train_bream_smelt[:5])

array(['Bream', 'Smelt', 'Bream', 'Bream', 'Bream'], dtype=object)

In [124]:
lr.predict_proba(train_bream_smelt[:5])

array([[0.99797409, 0.00202591],
       [0.02304883, 0.97695117],
       [0.99584843, 0.00415157],
       [0.98787711, 0.01212289],
       [0.99811716, 0.00188284]])

In [125]:
lr.classes_

array(['Bream', 'Smelt'], dtype=object)

In [126]:
lr.coef_

array([[-0.35504632, -0.50711944, -0.58428728, -0.58428728, -0.89668429,
        -0.64562519]])

### 로지스틱 다중분류 해보기

In [131]:
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target)
print(lr.score(train_scaled, train_target))

0.9327731092436975


C:\Users\2-22\Desktop\DMF\ML\venv\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [133]:
print(lr.score(test_scaled, test_target))

0.925


In [135]:
import numpy as np
np.round(lr.predict_proba(test_scaled[:5]), decimals=3)

array([[0.   , 0.014, 0.842, 0.   , 0.135, 0.007, 0.003],
       [0.   , 0.003, 0.044, 0.   , 0.007, 0.946, 0.   ],
       [0.   , 0.   , 0.034, 0.934, 0.015, 0.016, 0.   ],
       [0.011, 0.034, 0.305, 0.006, 0.567, 0.   , 0.076],
       [0.   , 0.   , 0.904, 0.002, 0.089, 0.002, 0.001]])

In [137]:
lr.predict_proba(test_scaled[:5])

array([[7.15677797e-06, 1.35523416e-02, 8.41952847e-01, 3.04290984e-04,
        1.34839427e-01, 6.77873955e-03, 2.56519794e-03],
       [7.00706104e-09, 2.59742704e-03, 4.40982995e-02, 3.29538672e-05,
        7.37357270e-03, 9.45892423e-01, 5.31709294e-06],
       [1.87019768e-05, 2.80057826e-06, 3.40026985e-02, 9.34383819e-01,
        1.52274568e-02, 1.63347111e-02, 2.98118188e-05],
       [1.08847164e-02, 3.39566550e-02, 3.05427047e-01, 6.45094490e-03,
        5.66821648e-01, 7.26113072e-05, 7.63863772e-02],
       [4.47372456e-06, 3.68054531e-04, 9.04258922e-01, 2.35657192e-03,
        8.92019896e-02, 2.47024447e-03, 1.33974352e-03]])

In [139]:
lr.predict(test_scaled[:5])

array(['Perch', 'Smelt', 'Pike', 'Roach', 'Perch'], dtype=object)

In [140]:
lr.classes_

array(['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish'],
      dtype=object)

In [146]:
decision = lr.decision_function(test_scaled[:5])
decision
# 생선에 대한 z값 모음
# 아직 확률이 아님.

array([[ -6.50722805,   1.03902669,   5.16819136,  -2.7573035 ,
          3.33655199,   0.34625853,  -0.62549701],
       [-10.87900739,   1.94410616,   4.77600603,  -2.42306185,
          2.98748716,   7.84171365,  -4.24724376],
       [ -4.33809947,  -6.23690278,   3.16746647,   6.48091388,
          2.36412675,   2.43431894,  -3.87182378],
       [ -0.69143987,   0.44628536,   2.64291145,  -1.21457289,
          3.2612452 ,  -5.70143412,   1.25700487],
       [ -6.40138602,  -1.9913762 ,   5.8152637 ,  -0.13464405,
          3.49905131,  -0.08753491,  -0.69937384]])

In [150]:
# 소프트맥스를 사용하여 z값을 하나의 확률로 계산
# 다중분류에서 사용
from scipy.special import softmax
proba = softmax(decision, axis=1)
np.round(proba, decimals=3)
# 근데 위에 lr.predict_proba(test_scaled[:5])값이랑 똑같다?
# ㅇㅇ 파이썬이 자동으로 소프트 맥스 사용해줌 ㅅㄱ

array([[0.   , 0.014, 0.842, 0.   , 0.135, 0.007, 0.003],
       [0.   , 0.003, 0.044, 0.   , 0.007, 0.946, 0.   ],
       [0.   , 0.   , 0.034, 0.934, 0.015, 0.016, 0.   ],
       [0.011, 0.034, 0.305, 0.006, 0.567, 0.   , 0.076],
       [0.   , 0.   , 0.904, 0.002, 0.089, 0.002, 0.001]])